<a href="https://colab.research.google.com/github/Sirabhop/CLM/blob/master/CLM_Administration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade gspread

In [ ]:
!pip install gspread-pandas

In [127]:
import pandas as pd
import os
import gspread
import datetime
import gspread_pandas

####**Connect google spreadsheet**

In [81]:
gc = gspread.service_account('/content/drive/My Drive/My CLM project/Machine Learning/credential/infinite-deck-288214-a013e2afcf8a.json')
sheet = gc.open('CLM Note')

worksheet_Admin = sheet.worksheet('Admin')
Admin = pd.DataFrame(worksheet_Admin.get_all_records())

In [70]:
worksheet_Admin

<Worksheet 'Admin' id:385677>

####**Call Report from telesales**

In [3]:
os.chdir("/content/drive/My Drive/My CLM project/Machine Learning")

In [4]:
usecols = range(1,27)
CallReport = pd.read_excel("Call Report CLM Date 01-30 Aug 2020.xlsx", header = 2,
                           usecols = usecols)

CallReport.iloc[0, 6:17].values

In [7]:
CallReport.rename(columns = {'ติดต่อได้':'สมัครบริการ Offer 1', 'Unnamed: 8':'% Success Offer 1', 'Unnamed: 9':'สมัครบริการ Offer 2', 
                             'Unnamed: 10':'% Success Offer 2', 'Unnamed: 11':'Total สมัครบริการ', 'Unnamed: 12':'Call Back', 
                             'Unnamed: 13':'Bad Lead', 'Unnamed: 14':'Follow Up', 'Unnamed: 15':'สนใจแต่ติดเงื่อนไข', 'Unnamed: 16':'ปฏิเสธ', 
                             'Unnamed: 17':'After Sale', 'Unnamed: 18':'สายหลุดระหว่างสนทนา', 'Campaign No.':'Key'}, inplace = True)
CallReport.drop(index = 0, axis = 0, inplace = True)
CallReport.drop(['Campaign Name', 'LIFESTAGE', 'LIFESTYLE'], axis = 1, inplace = True)

####**Jira Ticket**

In [42]:
Jira = pd.read_excel('CLM Daily Status Updates for Request Portal as of 9_2_2020.xlsx', sheet_name = 'JIRA Ticket')
Jira.drop(['Summary', 'Assignee', 'Resolution', 'Reject Reason', 'Reject Reason (Others)',
           'Waiting Reason', 'Resolved', 'Swimlane', 'CLM Code', 'Created', 'Updated',
           'Resolved Date', 'Reference Key'], axis = 1, inplace = True)

In [90]:
Jira.head()

,Key,Status,Lifestage Name,Lifestyle Name,Request Channel,CLM Request Type,Campaign Channel,Total Leads,Create Date,Telesale Project Name,Preferred Campaign Launch,Product Catelog
0,CLM-7118,00. Backlog,Experienced Professional Rural,Low Usage,Request Portal,Campaign,SMS Broadcast,NaN,2020-09-02,NaN,NaT,NaN
1,CLM-7117,00. Backlog,Family with Grown-up Kid,Education,Request Portal,Campaign,Call-Center,NaN,2020-09-02,UTMH4G+SharePlan1099OnPosPreMGMNP,NaT,NaN
2,CLM-7116,00. Backlog,University,Music,Request Portal,Campaign,SMS Campaign,NaN,2020-09-02,NaN,NaT,NaN
3,CLM-7115,00. Backlog,University,Movie Streamer,Request Portal,Campaign,SMS Broadcast,NaN,2020-09-02,NaN,NaT,NaN
4,CLM-7114,00. Backlog,Heavy Duty,Digital Trendy,Request Portal,Campaign,SMS Broadcast,NaN,2020-09-02,NaN,NaT,NaN


In [110]:
Jira.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7009 entries, 0 to 7008
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   Key                        7009 non-null   object        
 1   Status                     6585 non-null   object        
 2   Lifestage Name             6549 non-null   object        
 3   Lifestyle Name             6585 non-null   object        
 4   Request Channel            7009 non-null   object        
 5   CLM Request Type           7008 non-null   object        
 6   Campaign Channel           6844 non-null   object        
 7   Total Leads                3178 non-null   float64       
 8   Create Date                6585 non-null   datetime64[ns]
 9   Telesale Project Name      3424 non-null   object        
 10  Preferred Campaign Launch  3028 non-null   datetime64[ns]
 11  Product Catelog            5000 non-null   object        
dtypes: dat

In [108]:
Jira['Create Date'] = pd.to_datetime(Jira['Create Date'], format = '%Y-%m-%d')

####**Integrate Jira, Call Report, & Spreadsheet**

In [9]:
df = Jira.merge(CallReport, on = 'Key', how = 'outer')

In [11]:
df2 = df[['Key', 'Status', 'Lifestyle Name', 'Lifestage Name', 'CLM Request Type', 'Total Leads', 'Created', 'Telesale Project Name', 'Preferred Campaign Launch',
   '%Success', 'Grand Total']]

In [12]:
df2['Progress'] = df['Grand Total']/df['Total Leads']

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
to_upload = pd.merge(Admin, df2[['Status', 'Total Leads', 'Preferred Campaign Launch', '%Success', 
                'Grand Total', 'Progress', 'Key']], how = 'inner', on = 'Key')

In [118]:
to_upload['Preferred Campaign Launch'] = to_upload['Preferred Campaign Launch'].astype(str)

In [119]:
to_upload.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 21 entries, 0 to 20
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Key                        21 non-null     object 
 1   C                          21 non-null     object 
 2   Type                       21 non-null     object 
 3   Campaign                   21 non-null     object 
 4   CLM catalog                21 non-null     object 
 5   KMS                        21 non-null     object 
 6   SEP                        21 non-null     object 
 7   OCT                        21 non-null     object 
 8   NOV                        21 non-null     object 
 9   DEC                        21 non-null     object 
 10  Total Lead                 21 non-null     object 
 11  Actual Lead                21 non-null     object 
 12  Start                      21 non-null     object 
 13  End                        21 non-null     object 
 

In [ ]:
gspread_pandas.Spread.df_to_sheet(df = to_upload)